# Registering a cell in an anatomical reconstruction

Here, we place a cell in barrel cortex and compute an anatomical realization. This reconstruction specifies where synapses are located. Afterwards, we activate the synapses based on experimental measurements. Next, we import the simulation result to analyze it. Whenever necessary, dask and distributed is used to parallelize the computation.

In [ ]:
import Interface as I
from getting_started import getting_started_dir
import os
db = I.DataBase('{}/labs/getting_started_db/'.format(os.environ['HOME']))

trying to connect to distributed locking server {'type': 'file'}
[WARNING] distributed_lock: Using file based locking. Please be careful on nfs mounts as file based locking has issues in this case.
[INFO] ISF: Current version: heads/publish+0.gc832a64c.dirty
[INFO] ISF: Current pid: 221081
[INFO] mechanisms: Loading mechanisms:


--No graphics will be displayed.





[INFO] ISF: Loaded modules with __version__ attribute are:
IPython: 8.12.3, Interface: heads/publish+0.gc832a64c.dirty, PIL: 8.2.0, _csv: 1.0, _ctypes: 1.1.0, _curses: b'2.2', _decimal: 1.70, argparse: 1.1, attr: 20.3.0, backcall: 0.2.0, blake3: 0.3.3, blosc: 1.10.2, bluepyopt: 1.9.126, bottleneck: 1.3.2, cffi: 1.14.3, click: 7.1.2, cloudpickle: 1.6.0, colorama: 0.4.4, comm: 0.2.1, csv: 1.0, ctypes: 1.1.0, cycler: 0.10.0, cytoolz: 0.11.0, dash: 2.9.3, dask: 2.30.0, dateutil: 2.8.2, deap: 1.3, debugpy: 1.8.0, decimal: 1.70, decorator: 4.4.2, distributed: 2.30.1, distutils: 3.8.5, executing: 2.0.1, filelock: 3.0.12, flask: 1.1.2, flask_cors: 4.0.0, frozendict: 2.3.8, fsspec: 0.8.3, future: 0.18.2, gevent: 20.9.0, greenlet: 0.4.17, ipaddress: 1.0, ipykernel: 6.29.0, ipython_genutils: 0.2.0, ipywidgets: 7.5.1, itsdangerous: 1.1.0, jedi: 0.17.1, jinja2: 3.0.3, joblib: 1.3.2, json: 2.0.9, jupyter_client: 8.6.0, jupyter_core: 5.7.1, kiwisolver: 1.3.0, llvmlite: 0.34.0, logging: 0.5.1.2, ma

## Step 1: Registering the cell morphology in the desired reference frame

From external resources (e.g. the NeuroMorph pipeline), you need a hoc-file with the dendrite morphology you want to place in your anatomical model of a brain area of interest. The coordinates in the hoc morphology file need to be anchored at the desired location.

As an example, we will use a morphology of a Layer-5 Pyramidal Tract Neuron (L5PT) whose coordinates are anchored in the C2 column of a rat barrel cortex (BC)

In [ ]:
I.os.listdir(I.os.path.join(getting_started_dir, 'anatomical_constraints'))

['86_L5_CDK20041214_nr3L5B_dend_PC_neuron_transform_registered_C2center_synapses_20150504-1611_10389',
 '86_L5_CDK20041214_nr3L5B_dend_PC_neuron_transform_registered_C2center.hoc',
 '86_L5_CDK20041214_nr3L5B_dend_PC_neuron_transform_registered_C2center.swc',
 '86_L5_CDK20041214_nr3L5B_dend_PC_neuron_transform_registered_C2center_scaled_diameters.hoc']

For building the anatomical model, we use `86_L5_CDK20041214_nr3L5B_dend_PC_neuron_transform_registered_C2center.hoc`, for the simulation of the evoked activity, we need to use the morphology file with scaled apical trunk.

In [ ]:
path_to_hoc = I.os.path.join(getting_started_dir, 'anatomical_constraints', \
                    '86_L5_CDK20041214_nr3L5B_dend_PC_neuron_transform_registered_C2center.hoc')
path_to_scaled_hoc = I.os.path.join(getting_started_dir, 'anatomical_constraints', \
                    '86_L5_CDK20041214_nr3L5B_dend_PC_neuron_transform_registered_C2center_scaled_diameters.hoc')

We copy these files to our DataBase. 

In [ ]:
if not 'anatomical_constraints' in db.keys():
    db.create_managed_folder('anatomical_constraints')
    I.shutil.copy(path_to_hoc, db['anatomical_constraints'])
    I.shutil.copy(path_to_scaled_hoc, db['anatomical_constraints'])

## Step 2: calculate positions of anatomical synapses with respect to the registered morphology

We use [`singlecell_input_mapper`](../singlecell_input_mapper/readme.md) to create an anatomical model of how that cell is integrated in the brain are (here: the barrel cortex). For more information on how this is done, see [Egger et al. 2014](https://www.frontiersin.org/articles/10.3389/fnana.2014.00129/full). This module creates an anatomical reconstruction of axo-dendritic connections depending on the spatial distribution of cells, bouton density, and anatomical constraints of post-synaptic targets of the postsynaptic cell, such as morphology and synapse density. To do so, it needs the following input:
1. The neuron morphology (the `.hoc` file that we already copied over)
2. Density of total Post-Synaptic Target sites (PST) across the dendritic tree of the postynaptic cell (used for normalization)
3. Anatomical constraints of PSTs: the amount of synapses per length unit and area unit, depending on pre- and post-synaptic celltype, and the location along the dendritic tree of the postsynaptic cell. These are normalized using the previously mentioned PST densities.
4. Bouton densities across the entire brain area of interest.
5. Spatial distribution of cells, depending on their type.

Under the hood, 50 anatomical realizations will be computed that are congruent with the input criteria. From this distribution of anatomical realizations, the one that is closest to the average is chosen, which can be refered to as a "representative realization". The result is saved in the same folder as the hoc morphology. This takes about 4 hrs to compute, but you can continue with a precomputed result.

To compute it yourself, run the cell below. To copy a precomputed result, you can skip to the next code cell

In [ ]:
# Computing it yourself (this takes some time): --------------------
celltype = 'L5tt'  # Layer 5 thick-tufted, aka L5PT (pyramidal tract), aka L5ET (extratelencephalic)
path = db['anatomical_constraints'].join(
    '86_L5_CDK20041214_nr3L5B_dend_PC_neuron_transform_registered_C2center.hoc')
with I.silence_stdout:  # Silence output
    I.map_singlecell_inputs(
        cellName=path, 
        cellTypeName=celltype)

# adapt the path, if you have generated a new anatomical model
path_to_anatomical_model = db['anatomical_constraints'].join(
    '86_L5_CDK20041214_nr3L5B_dend_PC_neuron_transform_registered_C2center_synapses_20150504-1611_10389')

In [ ]:
# Pre-computed result: --------------------
from distutils.dir_util import copy_tree
dirname = '86_L5_CDK20041214_nr3L5B_dend_PC_neuron_transform_registered_C2center_synapses_20150504-1611_10389'
path_to_anatomical_model = I.os.path.join(getting_started_dir, 'anatomical_constraints', dirname, dirname)
silent = copy_tree(path_to_anatomical_model, db['anatomical_constraints'])

**What does the result of the map_singlecell_inputs script look like?**

In the directory generated by the singlecell_input_mapper, there are the following files:

In [ ]:
db['anatomical_constraints'].ls()

['Loader.json',
 '86_L5_CDK20041214_nr3L5B_dend_PC_neuron_transform_registered_C2center_synapses_20150504-1611_10389.syn',
 'presynaptic_somata.sh~',
 '86_L5_CDK20041214_nr3L5B_dend_PC_neuron_transform_registered_C2center_summary_20150504-1611_10389.csv',
 'total_synapses',
 '86_L5_CDK20041214_nr3L5B_dend_PC_neuron_transform_registered_C2center.hoc',
 'presynaptic_somata',
 'NumberOfConnectedCells.csv',
 'apical_synapses',
 '86_L5_CDK20041214_nr3L5B_dend_PC_neuron_transform_registered_C2center_synapses_20150504-1611_10389.con',
 '86_L5_CDK20041214_nr3L5B_dend_PC_neuron_transform_registered_C2center_scaled_diameters.hoc',
 'presynaptic_somata.sh',
 'metadata.json',
 'soma_synapses',
 'basal_synapses']

The most important files for making single-cell simulations are the `.con` file and the `.syn` file. These files are the relevant output of the `SingleCellMapper` for simulations of evoked activity.

The `.con` file maps presynaptic cells to a synapse. Not just a celltype, but all individual cells with a specific `cell ID` are mapped to an individual synapse with `synase ID`.

In [ ]:
con_file = db['anatomical_constraints'].get_file('.con')
con_file_path = db['anatomical_constraints'].join(con_file)
con_file_path

'/gpfs/soma_fs/home/meulemeester/labs/getting_started_db/anatomical_constraints/86_L5_CDK20041214_nr3L5B_dend_PC_neuron_transform_registered_C2center_synapses_20150504-1611_10389.con'

In [ ]:
with open(con_file_path) as f:
    print(f.read()[:300])

# Anatomical connectivity realization file; only valid with synapse realization:
# 86_L5_CDK20041214_nr3L5B_dend_PC_neuron_transform_registered_C2center_synapses_20150504-1611_10389.syn
# Type - cell ID - synapse ID

L6cc_A3	0	0
L6cc_A3	1	1
L6cc_A3	2	2
L6cc_A3	3	3
L6cc_A3	4	4
L6cc_A3	4	5
L6cc_A3	5	6


The `.syn` file specifies the exact position of each synapse on the hoc morphology:

In [ ]:
syn_file_path = db['anatomical_constraints'].join(db['anatomical_constraints'].get_file('.syn'))
with open(syn_file_path) as f:
    print(f.read()[:300])

# Synapse distribution file
# corresponding to cell: 86_L5_86_L5_CDK20041214_nr3L5B_dend_PC_neuron_transform_registered_C2center
# Type - section - section.x

VPM_E1	112	0.138046479525
VPM_E1	130	0.305058053119
VPM_E1	130	0.190509288017
VPM_E1	9	0.368760777084
VPM_E1	110	0.0
VPM_E1	11	0.120662910562


Here, section referes to the ID od the section in the cell object. x specifies, where along that section the synapse is placed. If x is 0, this is the beginning of the section, if x is one, this is the end of the section.

The subfolders in the directory contain Amira landmark files:
 - locations of presynaptic_somata by celltype in the folder `presynaptic_somata`
 - locations of synapses in the folders `total_synapses`, `soma_synapses`, `basal_synapses`, `apical_synapses`

### How to parallelize the generation of anatomical models?

We can use a distributed [`distributed.Client`](https://distributed.dask.org/en/latest/client.html) to create our `.syn` and `.con` files for one morphology per process:

In [ ]:
client = I.get_client(timeout=10)

In [ ]:
import psutil
print(psutil.cpu_count(logical=True))

48


In [ ]:
morphology_paths = [db['anatomical_constraints'].join(f) for f in db['anatomical_constraints'].listdir() if f.endswith('.hoc')]
morphology_paths

['/gpfs/soma_fs/home/meulemeester/labs/getting_started_db/anatomical_constraints/86_L5_CDK20041214_nr3L5B_dend_PC_neuron_transform_registered_C2center.hoc',
 '/gpfs/soma_fs/home/meulemeester/labs/getting_started_db/anatomical_constraints/86_L5_CDK20041214_nr3L5B_dend_PC_neuron_transform_registered_C2center_scaled_diameters.hoc']

This takes as long as creating an anatomical realization for a single cell, but thanks to parallellization, not much longer than that. At least, until the amount of cells exceed the amount of available threads.

In [ ]:
delayed_map_singlecell_inputs = I.dask.delayed(I.map_singlecell_inputs)  # make the single cell mapper a delayed function
delayeds = [delayed_map_singlecell_inputs(p, 'L5tt') for p in morphology_paths] # call it with the morphologies
delayeds = I.dask.delayed(delayeds) # bundle everything in one delayed object
futures = client.compute(delayeds)  # compute the result
# visualize the progress
I.distributed.progress(futures)

VBox()

You can restart the client if you have already started the computation, but don't want to compute this right now

In [ ]:
client.restart()

Client Scheduler: tcp://10.102.2.82:38786 Dashboard: http://10.102.2.82:38787/status,Cluster Workers: 24 Cores: 24 Memory: 98.30 GB


**Sidenotes**

`I.get_client()` returns a [`distributed.Client`](https://distributed.dask.org/en/latest/client.html) object, which can be used to execute a dask graph using all cores on the local machine. I.get_client() also "prepares" the cluster such that it fits the needs of doing single cell simulations: 
- It imports matplotlib on all workers and sets the backend to `'agg'`. This can be neccessary on compute servers that don't provide the necessary graphical libraries, which would crash otherwise. After this, you can create figures on the computeservers.
- If `I.get_client()` is called more than once, it does not generate a new [`distributed.Client`](https://distributed.dask.org/en/latest/client.html) object every time. Instead, it creates it when it is first called and returns the same client on all consecutive calls.
- You can set up a "real" cluster spanning hundreds of compute servers as described [here](http://distributed.readthedocs.io/en/latest/setup.html).

This setup now vocered all the necessary conditions to run a simulations where individual synapses are activated and evoke PSPs onto a taret neuron. See [the next notebook](./02_synaptic_stimulation.ipynb) on how to run the simulations.